# Script for crafting Adversarial Examples

In [19]:
import os

import utils.text_processing as tp

In [20]:
# get data from important word detector
%store -r important_words
%store -r sentence_packages

## Methed 2: Mispeeling

#### 1. Create modified Dataset

In [21]:
original_sentences = []
for sentence in sentence_packages:
    original_sentences.append(sentence['original_sentence'])

In [22]:
with open(os.path.join('data','birkbeck_corpera_typo.txt'), 'r') as f:
    typods = f.readlines()
    
typodict = {}
for index, item in enumerate(typods):
    if typods[index][0] == '$':
        word = typods[index].rstrip().strip('$')
        typo = typods[index+1].rstrip()
        typodict.update({word : typo})

In [23]:
modified_words = []
for word in important_words:
    modified_words.append(tp.to_typo(typodict, word))

In [24]:
modified_sentences = tp.generate_modified_sentences(original_sentences, important_words, modified_words)

#### 2. Import BERT Model

In [25]:
import torch
import random
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model.eval();

#### 3. Prediction

In [26]:
original_predictions = []
for sentence in original_sentences:
    original_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

In [27]:
modified_predictions = []
for sentence in modified_sentences:
    modified_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

#### 4. Creation of Dataset

In [30]:
ds = pd.DataFrame(list(zip(original_sentences, original_predictions, modified_sentences, modified_predictions)),
                 columns = ['original_sentences', 'original_predictions', 'modified_sentences', 'modified_predictions'])

In [32]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentences    435 non-null    object
 1   original_predictions  435 non-null    int64 
 2   modified_sentences    435 non-null    object
 3   modified_predictions  435 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 13.7+ KB


#### 5. Creation of adversarial Dataset

In [40]:
advds = ds[ds.original_predictions != ds.modified_predictions]

In [34]:
advds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 2 to 431
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentences    104 non-null    object
 1   original_predictions  104 non-null    int64 
 2   modified_sentences    104 non-null    object
 3   modified_predictions  104 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 4.1+ KB


In [38]:
 advds.to_json(r'data/adversarial_dataset_typos.json', orient='records')